### SUPPORT VECTOR MACHINE


In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader, SQLContext
from sqlalchemy import create_engine
import pandas as pd
spark= SparkSession.builder.getOrCreate()
spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [2]:
import urllib.request
url = 'https://github.com/LeonardoPalaciosPando1996/Database/blob/Master/hmp.parquet?raw=true'
filename = 'hmp.parquet'
urllib.request.urlretrieve(url, filename)

('hmp.parquet', <http.client.HTTPMessage at 0x1a408a25b20>)

In [3]:
df = spark.read.parquet('hmp.parquet')
df.createOrReplaceTempView('df')

In [4]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [5]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler

indexer = StringIndexer(inputCol="class", outputCol="label")
encoder = OneHotEncoder(inputCol="label", outputCol="labelVec")
vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [6]:
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [7]:
df.createOrReplaceTempView('df')
df_two_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")

In [8]:
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [9]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer,lsvc])

In [10]:
model = pipeline.fit(df_train)

In [11]:
prediction = model.transform(df_train)

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(prediction)

0.9350597289094003

In [13]:
prediction = model.transform(df_test)

In [14]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(prediction)

0.9368490437438509